# **Ising Model in 2D**

### [Go back to index](./index.ipynb)

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

<p style="text-align: justify;font-size:15px">  
    This notebook demonstrates the Ising model for a two dimensional system. Ising model is a
    mathematical model of ferromagnetism in statistical mechanics, which is named after the
    physicist Ernst Ising.
</p>

<p style="text-align: justify;font-size:15px">
    In a magnetic material system, each atom can be in spin up (+) and down (-) states. 
    The energy of a configuration can be described by a Hamiltonian function, which is 
    shown below:
</p>

$\large H = - \dfrac{1}{2} \sum_{i,j} J\sigma_i \sigma_j $ (1)

<p style="text-align: justify;font-size:15px">
    where $J_{ij}$ the strength of exchange interaction. Each atom only interects with its
    nearest-neighbor. Each $\sigma_i$ can be up (+1) and down (-1). Here, we are using 
    Monte Carlo simulation to study the spin configurations of the material systems.
</p>

## **Monte Carlo agorithm**

<p style="text-align: justify;font-size:15px">
<ol>
    <li>Start with some spin configuration.</li>
    <li>Randomly pick a site and consider flipping the spin over on that site.</li>
    <li>Compute energy for that preturbation.</li>
    <li>If $\delta E<0$ accept perturbation, if $\delta E>0$ accept perturbation with
        probability $exp\left[\frac{-\delta E}{kT}\right]$. </li>
    <li>Go back to step 2. </li>
</ol>    
</p>

## **Instruction for using the notebooks**

<p style="text-align: justify;font-size:15px">
    The figure on the left shows the spin configurations. Yellow pixel represents one spin 
    up and deep blue represents one spin down. You can change the exchange interaction parameter 
    J by tunning the slider. The "N" slider defines how many spins in one dimension. 
    The figure on the right shows the average spin changing with the Monte Carlo simulations. 
    By clicking "Run simulation", it will generate a random initial configuration and simulation
    one hundred steps. You can view the evolution of the simulations by the "Frame" slider.
</p>

In [ ]:
%reload_ext Cython
%matplotlib widget

In [ ]:
import numpy as np
from PIL import Image
from ipywidgets import interact, FloatSlider, Button, Output, IntSlider, VBox, HBox
import matplotlib.pyplot as plt
from time import sleep
import matplotlib.gridspec as gridspec
from scipy.ndimage import convolve

def random_spin_field(N, M):
    return np.random.choice([-1, 1], size=(N, M))

def display_spin_field(field):
    return Image.fromarray(np.uint8((field + 1) * 0.5 * 255))  # 0 ... 255

def display_ising_sequence(images):
    def _show(frame=(0, len(images) - 1)):
        return display_spin_field(images[frame])
    return interact(_show)

Jvalue = FloatSlider(value = -2.0, min = -2.0, max = 2.0, description = 'Exchange interaction J:',
                    style={'description_width': 'initial'})
Brun = Button(description='Run simulation')
Brun.style.button_color = 'lightgreen'

Frame = IntSlider(value=0, min=0, max=100, description="Frame", layout={'width':'800px'}, disabled=True)
Num = IntSlider(value=50, min=50, max=500, step=50, description="N")

In [ ]:
%%cython

cimport cython

import numpy as np
cimport numpy as np

from libc.math cimport exp
from libc.stdlib cimport rand
cdef extern from "limits.h":
    int RAND_MAX


@cython.boundscheck(False)
@cython.wraparound(False)
def cy_ising_step(np.int64_t[:, :] field, float J, float beta=0.4):
    cdef int N = field.shape[0]
    cdef int M = field.shape[1]
    cdef int n_offset, m_offset, n, m
    for n_offset in range(2):
        for m_offset in range(2):
            for n in range(n_offset, N, 2):
                for m in range(m_offset, M, 2):
                    _cy_ising_update(field, n, m, beta, J)
    return np.array(field)

@cython.boundscheck(False)
@cython.wraparound(False)
cdef _cy_ising_update(np.int64_t[:, :] field, int n, int m, float beta, float J):
    cdef int total = 0
    cdef int N = field.shape[0]
    cdef int M = field.shape[1]
    cdef int i, j
    for i in range(n-1, n+2):
        for j in range(m-1, m+2):
            if i == n and j == m:
                continue
            total += field[i % N, j % M]
    cdef float dE = J * field[n, m] * total
    if dE <= 0:
        field[n, m] *= -1
    elif exp(-dE * beta) * RAND_MAX > rand():
        field[n, m] *= -1

In [ ]:
def run_simulation(b):
    Brun.style.button_color = 'red'
    global images
    images = [random_spin_field(Num.value, Num.value)]
    
    x = np.arange(101);
    y1 = [];
    
    for i in range(100):
        images.append(cy_ising_step(images[-1].copy(), beta=0.4, J=Jvalue.value))
        
    fig.set_data(images[0])
    
    for i in images:
        y1.append(i.sum()*1.0/(Num.value*Num.value))
        
    line1.set_data(x, np.array(y1))
    
    Frame.value = 0
    Frame.disabled = False
      
    Brun.style.button_color = 'lightgreen'

def on_frame_change(b):
    fig.set_data(images[Frame.value])
    v1.set_data([Frame.value, Frame.value],[-1, 1])
    
def compute_total_energy(M, J):
    a = np.ones(np.shape(M));
    c = convolve(M, a, mode='constant')
    c = (c-M)*M*J
    return c.sum()

Brun.on_click(run_simulation);
Frame.observe(on_frame_change, names='value');

images = [random_spin_field(Num.value, Num.value)];

img = plt.figure(tight_layout=True, figsize=(8,5))
img.canvas.header_visible = False
gs = gridspec.GridSpec(5, 2)

ax1 = img.add_subplot(gs[:, 0])
ax2 = img.add_subplot(gs[1:4, 1])

fig = ax1.imshow(images[0])

line1, = ax2.plot([0], [0], 'r-')
v1 = ax2.axvline(x=0, c='black')

ax2.set_xlim([0, 100])
ax2.set_ylim([-1, 1])
ax2.set_title('Average spin', fontsize=12)

display(HBox([Jvalue, Num]), Brun, Frame)